In [1]:
import gurobipy as gp
from gurobipy import Model, GRB

#### Given:
### Tasks:
# T1 = (2,1)
# T2 = (3,1)
# T3 = (4,1)
# T4 = (5,1)
# T5 = (6,1)
# T6 = (10,1)
# T7 = (15,1)
# T8 = (25,1)

### Communication Costs:
# T1 - T3: 10
# T1 - T2: 7
# T2 - T4: 9
# T3 - T4: 13
# T3 - T5: 15
# T4 - T5: 8
# T5 - T6: 25
# T5 - T7: 2
# T7 - T8: 10

### Maximum Utilization:
# U_max = 1

### Number of modules:
# m = 2

In [2]:
### Maximum Utilization:
U_max = 1

### Number of modules:
num_modules = 2

In [3]:
# Given:
task_set = {
    "A": (100,1),
    "B": (100,1),
    "C": (100,1),
    "D": (100,1),
    "E": (100,1),
    "F": (100,1),
    "G": (100,1),
    "H": (100,1),
    "I": (100,1),
   "J": (100,1)
}

task_index = dict()
index = 0
for task in task_set:
    task_index[task] = index
    index += 1

tasks = []
for task in task_set:
    tasks.append(task)


modules = []
for i in range(1 , num_modules + 1):
    modules.append(f"m{i}")


task_utilization = dict()
for task in tasks:
    task_utilization[task] = task_set[task][1] / float(task_set[task][0])


#### Communication & Interference Cost
### Communication Costs
task_communication = [[0 for _ in range( len(tasks) )] for _ in range( len(tasks) )]

# Define communication costs
communication_costs = {
    (0,1): 3, # Task 1 - Task 2
    (0,3): 2,
    (0,7): 7,
    (0,8): 1,
    (1,2): 1,
    (2,3): 1,
    (3,4): 6,
    (4,5): 15,
    (5,6): 2,
    (5,7): 10,
    (7,8): 2
}

# Update the array with the communication costs
for (i, j), cost in communication_costs.items():
    task_communication[i][j] = cost

### Interference Costs
task_interference = [[0 for _ in range( len(tasks) )] for _ in range( len(tasks) )]

# Define Interference costs
Interference_costs = {
    (0,2): 3, # Task 1 - Task 3
    (0, 5): 100000, # Inf 
    (3, 5): 1,
    (4, 6): 4,
    (6, 7): 2
}

# Update the array with the Interference costs
for (i, j), cost in Interference_costs.items():
    task_interference[i][j] = cost

In [4]:
# Create Gurobi model
m = Model()

Set parameter Username
Set parameter LicenseID to value 2606738
Academic license - for non-commercial use only - expires 2026-01-06


In [5]:
### Variables: Binary Variable for each task for each module: x = 1 -> Task assigned to processor
x = m.addVars(tasks, modules, vtype=GRB.BINARY, name="x")

In [6]:
### Constraints:

## Constraint 1: Each task is assigned to one module
for task in tasks:
    m.addConstr( sum(x[task,module] for module in modules) == 1, name=f"assign_{task}")

## Constraint 2: For each module, sum of utilization of all tasks assigned to it is <= maximum allowable utilization
for module in modules:
    m.addConstr( sum( x[task,module]*task_utilization[task] for task in tasks) <= U_max, name = f"capacity_{module}")


## Constraint 3: TODO (Optional) -> Balanced Partitions
for module in modules:
    m.addConstr( sum( x[task,module] for task in tasks) == len(tasks)/2 )

In [7]:
cost = m.addVar(vtype = GRB.CONTINUOUS, name = 'cost')

# Add Variable z to simulate multiplication of two variables while keeping the problem linear
# x * Y -> z
# z <= x
# z <= y
# z >= x + y - 1
i = 1
z = dict()


for task_i in tasks:
    for task_j in tasks:
        if task_i == task_j:
            continue
        for module_k in modules:
            for module_l in modules:
                if module_k == module_l:
                    # Interference Cost:
                    temp_cost = task_interference[ task_index[task_i] ][ task_index[task_j] ]
                else:
                    # Communication Cost
                    temp_cost = task_communication[ task_index[task_i] ][ task_index[task_j] ]
                
                # Create z variable
                z[f"z{i}"] = m.addVar(vtype=GRB.BINARY, name=f'z{i}')
                # Add z constraints
                m.addConstr( z[f"z{i}"] <= x[task_i,module_k] )
                m.addConstr( z[f"z{i}"] <= x[task_j,module_l] )
                m.addConstr( z[f"z{i}"] >= x[task_i,module_k] + x[task_j,module_l] - 1 )
                
                cost += z[f"z{i}"] * temp_cost
                i += 1
        
        

m.setObjective(cost, GRB.MINIMIZE)

In [8]:
# Disable presolve
m.setParam("Presolve", 0)

# Optimize the model
m.optimize()

Set parameter Presolve to value 0
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11370H @ 3.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
Presolve  0

Optimize a model with 1094 rows, 381 columns and 2580 nonzeros
Model fingerprint: 0x7ce4bffe
Variable types: 1 continuous, 380 integer (380 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e+00, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 100047.00000
Variable types: 1 continuous, 380 integer (380 binary)

Root relaxation: objective 0.000000e+00, 102 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.

In [9]:
# Output the results
if m.status == GRB.OPTIMAL:
    # print("Optimal solution found!")
    print(f"Total Cost: {m.objVal}\n")
    for task in tasks:
        for module in modules:
            if x[task,module].X:
                print(f"{task[0]}: Assigned to Frame -> {module}")   
else:
    print("No optimal solution found.")

Total Cost: 15.0

A: Assigned to Frame -> m2
B: Assigned to Frame -> m2
C: Assigned to Frame -> m1
D: Assigned to Frame -> m1
E: Assigned to Frame -> m1
F: Assigned to Frame -> m1
G: Assigned to Frame -> m2
H: Assigned to Frame -> m1
I: Assigned to Frame -> m2
J: Assigned to Frame -> m2


In [10]:
print_cost = 0
for task_i in tasks:
    for task_j in tasks:
        if task_i == task_j:
            continue
        for module_k in modules:
            for module_l in modules:
                if module_k == module_l:
                    temp_cost = task_interference[ task_index[task_i]][ task_index[task_j] ]
                else:
                    temp_cost = task_communication[ task_index[task_i] ][ task_index[task_j] ]
                    
                if(x[task_i,module_k].X * x[task_j,module_l].X > 0):
                    if(temp_cost > 0):
                        if(module_k == module_l):
                            print("Interference :",temp_cost, f"-> task_i: {task_i}, task_j: {task_j}")
                        else:
                            print("Communication:",temp_cost, f"-> task_i: {task_i}, task_j: {task_j}")
                print_cost += x[task_i,module_k].X * x[task_j,module_l].X * temp_cost

print(f"Total Cost: {print_cost}")

Communication: 2 -> task_i: A, task_j: D
Communication: 7 -> task_i: A, task_j: H
Communication: 1 -> task_i: B, task_j: C
Interference : 1 -> task_i: D, task_j: F
Communication: 2 -> task_i: F, task_j: G
Communication: 2 -> task_i: H, task_j: I
Total Cost: 15.0
